In [1]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [2]:
import hail as hl
hl.init()

Running on Apache Spark version 2.4.1
SparkUI available at http://8c03623a3bed:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.28-61941242c15d
LOGGING: writing to /hail/large_vcf_filtering_annotation/hail-20191219-1219-0.2.28-61941242c15d.log


In [3]:
from hail.plot import show
from pprint import pprint
from bokeh.layouts import gridplot
hl.plot.output_notebook()

import numpy as np
import pandas as pd
from functools import reduce
from itertools import chain

Loading BokehJS ...

In [12]:
mt = hl.read_matrix_table('step1_output.mt')

In [9]:
mt_sample = mt.sample_rows(0.001)

In [10]:
eigenvalues, pcs, _ = hl.hwe_normalized_pca(mt_sample.GT)
mt_sample = mt_sample.annotate_cols(scores = pcs[mt_sample.s].scores)

2019-12-19 12:28:08 Hail: INFO: hwe_normalized_pca: running PCA using 7611 variants.
2019-12-19 12:28:32 Hail: INFO: pca: running PCA with 10 components...


In [11]:
p = hl.plot.scatter(mt_sample.scores[0],
                    mt_sample.scores[1],
                    label=mt_sample.s,
                    title='PCA', xlabel='PC1', ylabel='PC2')
show(p)

In [13]:
mt = mt.select_rows(mt.rsid, mt.info.AC, mt.info.AF, mt.info.AN)
mt = hl.split_multi_hts(mt)

In [14]:
mt.write('step2_output.mt')

2019-12-19 12:39:27 Hail: INFO: wrote matrix table with 8328985 rows and 151 columns in 591 partitions to step2_output.mt
